In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=d3aa2be20664a76776a349d9c94bd1bac306d3c464a4fd312af4c40f84c20950
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [ ]:
import os, re
import pandas as pd
import numpy as np
from scipy import stats
from pandasql import sqldf
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/My Drive/Drones/"
df = pd.read_csv(path+"cleaned_drones.csv", index_col = 0)

# Analyse data dispersion
## Analyse price first to detect and remove outliers

In [ ]:
print("Max price; ", round(df["price"].max()))
print("Mean price: ", round(df["price"].mean()))
print("Median: ", round(df["price"].median()))
print("Standard deviation: ", round(df["price"].std()))
print("Interquantile range: ", round(df["price"].quantile(0.75)-df["price"].quantile(0.25)))

Max price;  113999
Mean price:  3680
Trimmed mean price:  2238
Median:  989
Trimmed standard deviation: 5877
Interquantile range:  1610
Trimmed max price: 35000
Standard deviation:  9262


In [ ]:
fig = px.histogram(df, x="price", color_discrete_sequence=["darkcyan"], nbins=40)
fig.update_layout(autosize=False,width=1200,height=300, bargap=0.2)
fig.show()

### Drop the outliers.

As we can see there is a very far away outlier, compared to the rest which price is below $40, 000. To find if more products satisfy such price we can do a direct pandas search or a SQL query using pandasql.

In [ ]:
df[df["price"]>40000]

,link,brand,price,sales,stars,batteries_included,mini,flight_time,is_fpv,resolution,...,gps,remote_control,portable,helix_protector,autoreturn,full_turn,altitude_retention,dual_camara,distance,follow_func
106,https://www.mercadolibre.com.mx/drone-dji-mavi...,dji,113999,1,4.0,True,True,True,True,5,...,True,True,True,False,True,False,True,True,200,False
290,https://www.mercadolibre.com.mx/drone-dji-dji-...,dji,112669,50,4.0,True,True,True,False,4,...,True,True,False,False,True,False,True,False,200,True


Alternatevely we can use a SQL query.

In [ ]:
price_outliers = sqldf("SELECT * FROM df WHERE price > 40000")
print(price_outliers)

                                                link brand   price  sales  \
0  https://www.mercadolibre.com.mx/drone-dji-mavi...   dji  113999      1   
1  https://www.mercadolibre.com.mx/drone-dji-dji-...   dji  112669     50   

   stars  batteries_included  mini  flight_time  is_fpv  resolution  ...  gps  \
0    4.0                   1     1            1       1           5  ...    1   
1    4.0                   1     1            1       0           4  ...    1   

   remote_control  portable  helix_protector  autoreturn  full_turn  \
0               1         1                0           1          0   
1               1         0                0           1          0   

   altitude_retention  dual_camara  distance  follow_func  
0                   1            1       200            0  
1                   1            0       200            1  

[2 rows x 22 columns]


You can visit such products by printing the links.

In [ ]:
df.loc[106].link

'https://www.mercadolibre.com.mx/drone-dji-mavic-3-cine-premium-combo-con-dual-camara-51k-gris-58ghz-3-baterias/p/MLM18609039?pdp_filters=category:MLM149102#searchVariation=MLM18609039&position=12&search_layout=stack&type=product&tracking_id=9511bb7b-0f89-4519-a71d-900f8141864a'

In [ ]:
df.loc[290].link

'https://www.mercadolibre.com.mx/drone-dji-dji-phantom-4-pro-v20-11udh39r710318-v2-con-camara-c4k-blanco-1-bateria/p/MLM14159991?pdp_filters=category:MLM149102#searchVariation=MLM14159991&position=3&search_layout=stack&type=product&tracking_id=0a9e835e-a0d8-4010-917d-a84eb0a23340'

Now that we find such product we can dropped them:t o find the trimmed stats.

In [ ]:
df_trimmed_price = df.drop([106, 290])
print("Trimmed max price:", round(df_trimmed_price["price"].max()))
print("Trimmed mean price: ", round(stats.trim_mean(df["price"], 0.05)))
print("Trimmed median: ", round(df_trimmed_price["price"].median()))
print("Trimmed standard deviation:", round(df_trimmed_price["price"].std()))

Trimmed max price: 35000
Trimmed mean price:  2238
Trimmed median:  978
Trimmed standard deviation: 5877


In [ ]:
fig = px.histogram(df_trimmed_price, x="price", color_discrete_sequence=["darkcyan"], nbins=10)
fig.update_layout(autosize=False,width=1200,height=300, bargap=0.1)
fig.show()

In [ ]:
df_cheap = sqldf("SELECT * FROM df WHERE price < 5000")
fig = px.histogram(df_cheap, x="price", color_discrete_sequence=["darkcyan"], nbins=10)
fig.update_layout(autosize=False,width=1200,height=300, bargap=0.1)
fig.show()

## Then we can go for the rest of numeric metrics

Users seems to want batteries included, remote control, autoreturn, full turn, altitude retention, dual camara.

In [ ]:
df.shape

(471, 22)

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Sales", "Stars", "Camara resolution", "Flight distance"))
fig.add_trace(go.Histogram(x=df["sales"], nbinsx=5), row = 1, col = 1)
fig.add_trace(go.Histogram(x=df["stars"], nbinsx=6), row=1, col=2)
fig.add_trace(go.Histogram(x=df["resolution"]), row=2, col=1)
fig.add_trace(go.Histogram(x=df["distance"], nbinsx=8), row=2, col=2)
fig.update_layout(autosize=False,width=1000,height=600, bargap=0.5)
fig.show()

In [ ]:
fig = px.histogram(df, x="stars", color_discrete_sequence=["coral"], nbins=6)
fig.update_layout(autosize=False,width=600,height=300, bargap=0.1)
fig.show()

Portable, is fpv, gps, batteries included.

In [ ]:
fig = make_subplots(rows=2, cols=4, subplot_titles=("Batteries included", "Altitude retention", "Dual  camara",
                                                    "Follow function", "Full turn",  "Gps", "Portable", "FPV"))
fig.add_trace(go.Histogram(x=df["batteries_included"]), row = 1, col = 1)
fig.add_trace(go.Histogram(x=df["altitude_retention"]), row = 1, col= 2)
fig.add_trace(go.Histogram(x=df["dual_camara"]), row = 1, col= 3)
fig.add_trace(go.Histogram(x=df["follow_func"]), row = 1, col= 4)
fig.add_trace(go.Histogram(x=df["full_turn"]), row = 2, col = 1)
fig.add_trace(go.Histogram(x=df["gps"]), row = 2, col= 2)
fig.add_trace(go.Histogram(x=df["portable"]), row = 2, col= 3)
fig.add_trace(go.Histogram(x=df["is_fpv"]), row = 2, col= 4)
fig.update_layout(autosize=False,width=1200,height=600, bargap=0.7)
fig.show()

## Correlation

In [ ]:
fig = px.imshow(df_trimmed_price.corr(numeric_only=True))
fig.update_layout(autosize=False,width=1200,height=800)
fig.show()

# The 5 most selled.

In [100]:
#If we write df.nlargest(6, ["sales"], and head(6) we can note that the 6st product has aprox 1000 sales intead of 5000.
best_sellers= df.nlargest(5, ["sales"])
best_sellers.head()

,link,brand,price,sales,stars,batteries_included,mini,flight_time,is_fpv,resolution,...,gps,remote_control,portable,helix_protector,autoreturn,full_turn,altitude_retention,dual_camara,distance,follow_func
1,https://www.mercadolibre.com.mx/control-remoto...,yabely,732,5000,3.6,True,True,True,True,4,...,False,True,False,False,False,False,False,False,200,False
27,https://articulo.mercadolibre.com.mx/MLM-14895...,bohai,2057,5000,4.0,True,True,True,True,3,...,True,True,False,False,True,False,False,True,10000,True
37,https://www.mercadolibre.com.mx/mini-drone-axn...,axnen,891,5000,4.0,True,True,True,False,2,...,False,True,True,False,True,True,True,True,6000,False
146,https://articulo.mercadolibre.com.mx/MLM-14991...,axnen,891,5000,4.0,True,True,True,False,2,...,False,True,True,False,True,True,True,True,200,False
596,https://articulo.mercadolibre.com.mx/MLM-14621...,yabely,407,5000,4.0,True,True,True,True,4,...,False,True,True,False,False,False,False,False,200,False


In [103]:
# Show the whole information of best_sellers["link"] that is without trim the strings
for link in best_sellers["link"]:
  print(link)

https://www.mercadolibre.com.mx/control-remoto-drone-con-par-camara-4k-quadcopter-2bateria-color-negro1/p/MLM24084289?pdp_filters=category:MLM149102#searchVariation=MLM24084289&position=12&search_layout=stack&type=product&tracking_id=bf3dfce2-4775-455b-87fe-55be1960995c
https://articulo.mercadolibre.com.mx/MLM-1489545823-drone-con-camara-profesional-8k-a-control-remoto-gps-5ghz-_JM#position=26&search_layout=stack&type=item&tracking_id=bf3dfce2-4775-455b-87fe-55be1960995c
https://www.mercadolibre.com.mx/mini-drone-axnen-4k-xt9-3-battery-con-dual-camara-hd-24ghz-3-baterias-oscuro-gris/p/MLM26080871?pdp_filters=category:MLM149102#searchVariation=MLM26080871&position=8&search_layout=stack&type=product&tracking_id=bf3dfce2-4775-455b-87fe-55be1960995c
https://articulo.mercadolibre.com.mx/MLM-1499158704-dron-cuadricoptero-axnen-xt9-a-control-remoto-ccamara-4k-hd-_JM#position=47&search_layout=stack&type=item&tracking_id=28c60b3e-2922-4e2f-ac6f-22888af73d94
https://articulo.mercadolibre.com.mx/

In [106]:
print("Max price; ", round(best_sellers["price"].max()))
print("Mean price: ", round(best_sellers["price"].mean()))
print("Median: ", round(best_sellers["price"].median()))
print("Resolution: ", best_sellers["resolution"].median())
print("Wifi: ", best_sellers["wifi_freq"].mean())
print("Distance", best_sellers["distance"].median())

Max price;  2057
Mean price:  996
Median:  891
Resolution:  3.0
Wifi:  3.08
Distance 200.0


In [107]:
fig = make_subplots(rows=2, cols=4, subplot_titles=("Batteries included", "Altitude retention", "Dual  camara",
                                                    "Follow function", "Full turn",  "Gps", "Portable", "FPV"))
fig.add_trace(go.Histogram(x=best_sellers["batteries_included"]), row = 1, col = 1)
fig.add_trace(go.Histogram(x=best_sellers["altitude_retention"]), row = 1, col= 2)
fig.add_trace(go.Histogram(x=best_sellers["dual_camara"]), row = 1, col= 3)
fig.add_trace(go.Histogram(x=best_sellers["follow_func"]), row = 1, col= 4)
fig.add_trace(go.Histogram(x=best_sellers["full_turn"]), row = 2, col = 1)
fig.add_trace(go.Histogram(x=best_sellers["gps"]), row = 2, col= 2)
fig.add_trace(go.Histogram(x=best_sellers["portable"]), row = 2, col= 3)
fig.add_trace(go.Histogram(x=best_sellers["is_fpv"]), row = 2, col= 4)
fig.update_layout(autosize=False,width=1200,height=600, bargap=0.7)
fig.show()

# Best products

In [108]:
pd.set_option('display.max_colwidth', 50)
best_products = df.nlargest(25, ["sales", "stars"])
best_products.head()

,link,brand,price,sales,stars,batteries_included,mini,flight_time,is_fpv,resolution,...,gps,remote_control,portable,helix_protector,autoreturn,full_turn,altitude_retention,dual_camara,distance,follow_func
27,https://articulo.mercadolibre.com.mx/MLM-14895...,bohai,2057,5000,4.0,True,True,True,True,3,...,True,True,False,False,True,False,False,True,10000,True
37,https://www.mercadolibre.com.mx/mini-drone-axn...,axnen,891,5000,4.0,True,True,True,False,2,...,False,True,True,False,True,True,True,True,6000,False
146,https://articulo.mercadolibre.com.mx/MLM-14991...,axnen,891,5000,4.0,True,True,True,False,2,...,False,True,True,False,True,True,True,True,200,False
596,https://articulo.mercadolibre.com.mx/MLM-14621...,yabely,407,5000,4.0,True,True,True,True,4,...,False,True,True,False,False,False,False,False,200,False
1,https://www.mercadolibre.com.mx/control-remoto...,yabely,732,5000,3.6,True,True,True,True,4,...,False,True,False,False,False,False,False,False,200,False


In [110]:
print("Max price; ", round(best_products["price"].max()))
print("Mean price: ", best_products["price"].mean())
print("Median price: ", best_products["price"].median())
print("Resolution: ", best_products["resolution"].median())
print("Wifi: ", best_products["wifi_freq"].mean())
print("Distance", best_products["distance"].median())

Max price;  19999
Mean price:  2833.0
Median price:  891.0
Resolution:  4.0
Wifi:  4.272
Distance 200.0


In [ ]:
pd.set_option('display.max_colwidth', None)
best_products["link"].head(10)

1                      https://www.mercadolibre.com.mx/control-remoto-drone-con-par-camara-4k-quadcopter-2bateria-color-negro1/p/MLM24084289?pdp_filters=category:MLM149102#searchVariation=MLM24084289&position=12&search_layout=stack&type=product&tracking_id=bf3dfce2-4775-455b-87fe-55be1960995c
27                                                                                       https://articulo.mercadolibre.com.mx/MLM-1489545823-drone-con-camara-profesional-8k-a-control-remoto-gps-5ghz-_JM#position=26&search_layout=stack&type=item&tracking_id=bf3dfce2-4775-455b-87fe-55be1960995c
37            https://www.mercadolibre.com.mx/mini-drone-axnen-4k-xt9-3-battery-con-dual-camara-hd-24ghz-3-baterias-oscuro-gris/p/MLM26080871?pdp_filters=category:MLM149102#searchVariation=MLM26080871&position=8&search_layout=stack&type=product&tracking_id=bf3dfce2-4775-455b-87fe-55be1960995c
146                                                                                    https://articulo.mercadolibre.c

In [ ]:
fig = px.histogram(best_products, x="price", color_discrete_sequence=["darkcyan"], nbins=40)
fig.update_layout(autosize=False,width=1200,height=300, bargap=0.1)
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Sales", "Stars", "Camara resolution", "Flight distance"))
fig.add_trace(go.Histogram(x=best_products["sales"], nbinsx=8), row = 1, col = 1)
fig.add_trace(go.Histogram(x=best_products["stars"], nbinsx=8), row=1, col=2)
fig.add_trace(go.Histogram(x=best_products["resolution"]), row=2, col=1)
fig.add_trace(go.Histogram(x=best_products["distance"], nbinsx=8), row=2, col=2)
fig.update_layout(autosize=False,width=1200,height=600, bargap=0.7)
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=4, subplot_titles=("Batteries included", "Altitude retention", "Dual  camara",
                                                    "Follow function", "Full turn",  "Gps", "Portable", "FPV"))
fig.add_trace(go.Histogram(x=best_products["batteries_included"]), row = 1, col = 1)
fig.add_trace(go.Histogram(x=best_products["altitude_retention"]), row = 1, col= 2)
fig.add_trace(go.Histogram(x=best_products["dual_camara"]), row = 1, col= 3)
fig.add_trace(go.Histogram(x=best_products["follow_func"]), row = 1, col= 4)
fig.add_trace(go.Histogram(x=best_products["full_turn"]), row = 2, col = 1)
fig.add_trace(go.Histogram(x=best_products["gps"]), row = 2, col= 2)
fig.add_trace(go.Histogram(x=best_products["portable"]), row = 2, col= 3)
fig.add_trace(go.Histogram(x=best_products["is_fpv"]), row = 2, col= 4)
fig.update_layout(autosize=False,width=1200,height=600, bargap=0.7)
fig.show()

In [ ]:
fig = px.imshow(best_products.corr(numeric_only=True))
fig.update_layout(autosize=False,width=1000,height=800)
fig.show()

# By brand

In [117]:
brands = df["brand"].value_counts()
#brands.drop("generic", inplace = True)
brands.head(11)

brand
generic      154
dji           46
holy_tone     19
sjrc          16
binden        12
lsrc           8
axnen          8
general        7
deerc          6
drone          6
broadream      5
Name: count, dtype: int64

In [ ]:
pd.set_option('display.max_colwidth', 50)
dji = df[df["brand"] == "dji"]
dji.sort_values("sales", ascending = False).head(10)

,link,brand,price,sales,stars,batteries_included,mini,flight_time,is_fpv,resolution,...,gps,remote_control,portable,helix_protector,autoreturn,full_turn,altitude_retention,dual_camara,distance,follow_func
4,https://www.mercadolibre.com.mx/mini-drone-dji...,dji,11467,1000,4.6,True,True,True,True,4,...,True,True,True,False,True,True,True,False,200,False
39,https://www.mercadolibre.com.mx/mini-drone-dji...,dji,12399,1000,4.0,True,True,True,True,4,...,True,True,True,False,True,True,True,False,200,False
421,https://www.mercadolibre.com.mx/drone-dji-air-...,dji,23999,500,4.0,True,False,True,False,4,...,True,True,True,False,True,False,True,False,200,True
152,https://www.mercadolibre.com.mx/dji-mini-3-pro...,dji,19999,500,4.9,True,True,True,True,4,...,True,True,True,False,True,False,True,False,200,False
520,https://articulo.mercadolibre.com.mx/MLM-17635...,dji,18999,100,4.9,True,True,True,False,4,...,True,False,True,False,True,False,False,False,200,True
35,https://www.mercadolibre.com.mx/mini-drone-dji...,dji,20999,100,4.3,True,True,True,False,4,...,True,True,True,False,False,False,False,False,4000,False
45,https://www.mercadolibre.com.mx/mini-drone-dji...,dji,8999,100,4.0,True,True,True,False,3,...,True,True,True,True,True,False,True,False,200,True
175,https://articulo.mercadolibre.com.mx/MLM-18472...,dji,8299,100,4.0,True,True,True,False,7,...,False,False,True,False,False,False,False,False,200,False
11,https://www.mercadolibre.com.mx/dji-drone-mini...,dji,8299,100,5.0,True,True,True,False,7,...,True,True,True,False,False,False,False,False,18000,False
128,https://www.mercadolibre.com.mx/mini-drone-dji...,dji,8999,100,5.0,True,True,True,False,7,...,True,True,False,False,True,False,False,False,18000,False


In [136]:
print("Max price; ", round(dji["price"].max()))
print("Min price; ", round(dji["price"].min()))
print("Mean price: ", dji["price"].mean())
print("Median price: ", dji["price"].median())
print("Resolution: ", dji["resolution"].median())
print("Wifi: ", dji["wifi_freq"].mean())
print("Distance", dji["distance"].median())
print("Stars", dji["stars"].mean())
print("Sales", dji["sales"].sum())

Max price;  113999
Min price;  1348
Mean price:  21310.521739130436
Median price:  13249.5
Resolution:  4.0
Wifi:  4.1000000000000005
Distance 200.0
Stars 4.269565217391304
Sales 4174


In [144]:
# prompt: show me the link of the row of the dji dataset such that the column "price" has the third minimum value

print(dji.loc[dji["price"].nsmallest(3).index[2]]["link"])


https://articulo.mercadolibre.com.mx/MLM-634230535-nuevo-drone-dji-tello-smart-mini-camara-envio-gratis-msi-_JM#position=33&search_layout=stack&type=item&tracking_id=53d667ff-f6cd-4bb1-b4d9-1ddfff039923


In [131]:
holy_stone = df[df["brand"] == "holy_tone"]
print("Max price; ", round(holy_stone["price"].max()))
print("Mean price: ", holy_stone["price"].mean())
print("Median price: ", holy_stone["price"].median())
print("Resolution: ", holy_stone["resolution"].median())
print("Wifi: ", holy_stone["wifi_freq"].mean())
print("Distance", holy_stone["distance"].median())
print("Stars", holy_stone["stars"].mean())
print("Sales", holy_stone["sales"].sum())

Max price;  7844
Mean price:  2821.0
Median price:  1277.0
Resolution:  3.0
Wifi:  2.810526315789473
Distance 200.0
Stars 4.2105263157894735
Sales 1979


In [130]:
sjrc = df[df["brand"] == "sjrc"]
print("Max price; ", round(sjrc["price"].max()))
print("Mean price: ", sjrc["price"].mean())
print("Median price: ", sjrc["price"].median())
print("Resolution: ", sjrc["resolution"].median())
print("Wifi: ", sjrc["wifi_freq"].mean())
print("Distance", sjrc["distance"].median())
print("Stars", sjrc["stars"].mean())
print("Sales", sjrc["sales"].sum())

Max price;  9999
Mean price:  6694.625
Median price:  7402.0
Resolution:  4.0
Wifi:  4.0375
Distance 200.0
Stars 4.2125
Sales 1713


In [133]:
binden = df[df["brand"] == "binden"]
print("Max price; ", round(binden["price"].max()))
print("Mean price: ", binden["price"].mean())
print("Median price: ", binden["price"].median())
print("Resolution: ", binden["resolution"].median())
print("Wifi: ", binden["wifi_freq"].mean())
print("Distance", binden["distance"].median())
print("Stars", binden["stars"].mean())
print("Sales", binden["sales"].sum())

Max price;  7499
Mean price:  4015.6666666666665
Median price:  3749.0
Resolution:  4.0
Wifi:  3.6999999999999997
Distance 200.0
Stars 4.041666666666667
Sales 825


In [134]:
axnen = df[df["brand"] == "axnen"]
print("Max price; ", round(axnen["price"].max()))
print("Mean price: ", axnen["price"].mean())
print("Median price: ", axnen["price"].median())
print("Resolution: ", axnen["resolution"].median())
print("Wifi: ", axnen["wifi_freq"].mean())
print("Distance", axnen["distance"].median())
print("Stars", axnen["stars"].mean())
print("Sales", axnen["sales"].sum())

Max price;  2096
Mean price:  1282.375
Median price:  993.5
Resolution:  4.0
Wifi:  3.375
Distance 200.0
Stars 4.0875
Sales 12705
